# Fama-macbeth回归

In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import lstsq
import statsmodels.api as sm
from finance_byu.fama_macbeth import fama_macbeth, fm_summary

# 原始数据

In [2]:
data = pd.read_csv('ff_data.csv')
data['month'] = data['month'].astype(str)
data

,month,SL,SM,SH,BL,BM,BH,mkt-rf,smb,hml
0,202201,-13.3312,-9.0569,-1.0018,-7.9554,-2.8507,5.2226,-6.25,-5.94,12.75
1,202202,0.0517,1.2208,2.2370,-3.5601,0.0278,0.3395,-2.29,2.23,3.04
2,202203,1.7806,1.0372,0.5629,4.3278,1.9104,1.9377,3.05,-1.60,-1.80
3,202204,-13.5805,-8.5466,-5.5740,-10.5954,-6.6579,-6.2179,-9.46,-1.41,6.19
4,202205,-4.4206,1.6393,4.1352,-2.6261,3.8821,5.6393,-0.34,-1.85,8.41
5,202206,-4.3163,-6.6654,-11.2053,-7.2567,-8.9137,-12.2993,-8.43,2.09,-5.97
6,202207,13.4122,10.8013,8.6857,11.4381,5.1174,7.9978,9.56,2.78,-4.08


In [3]:
unstack = data.drop(columns=['mkt-rf','smb','hml'])
unstack = pd.DataFrame(unstack.set_index('month').unstack()).reset_index()
unstack.columns = ['port','month','ret']
unstack.head(10)

,port,month,ret
0,SL,202201,-13.3312
1,SL,202202,0.0517
2,SL,202203,1.7806
3,SL,202204,-13.5805
4,SL,202205,-4.4206
5,SL,202206,-4.3163
6,SL,202207,13.4122
7,SM,202201,-9.0569
8,SM,202202,1.2208
9,SM,202203,1.0372


In [4]:
test = pd.merge(unstack, data[['month','mkt-rf','smb','hml']], how='left', on='month')
test

,port,month,ret,mkt-rf,smb,hml
0,SL,202201,-13.3312,-6.25,-5.94,12.75
1,SL,202202,0.0517,-2.29,2.23,3.04
2,SL,202203,1.7806,3.05,-1.60,-1.80
3,SL,202204,-13.5805,-9.46,-1.41,6.19
4,SL,202205,-4.4206,-0.34,-1.85,8.41
5,SL,202206,-4.3163,-8.43,2.09,-5.97
6,SL,202207,13.4122,9.56,2.78,-4.08
7,SM,202201,-9.0569,-6.25,-5.94,12.75
8,SM,202202,1.2208,-2.29,2.23,3.04
9,SM,202203,1.0372,3.05,-1.60,-1.80


In [5]:
temp = test[test['port'] == 'SL']
temp

,port,month,ret,mkt-rf,smb,hml
0,SL,202201,-13.3312,-6.25,-5.94,12.75
1,SL,202202,0.0517,-2.29,2.23,3.04
2,SL,202203,1.7806,3.05,-1.60,-1.80
3,SL,202204,-13.5805,-9.46,-1.41,6.19
4,SL,202205,-4.4206,-0.34,-1.85,8.41
5,SL,202206,-4.3163,-8.43,2.09,-5.97
6,SL,202207,13.4122,9.56,2.78,-4.08


# 测试

In [6]:
lstsq(temp[['mkt-rf','smb','hml']],temp[['ret']],rcond=None)
# return beta, residual?? rank, s??

(array([[ 0.99720375],
        [ 0.95305656],
        [-0.21570242]]),
 array([5.67622526]),
 3,
 array([22.2840496 , 13.65957009,  4.38306722]))

In [7]:
res = sm.OLS(temp['ret'],temp[['mkt-rf','smb','hml']]).fit()
res.summary()

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 7 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                    ret   R-squared (uncentered):                   0.990
Model:                            OLS   Adj. R-squared (uncentered):              0.983
Method:                 Least Squares   F-statistic:                              135.7
Date:                Sat, 17 Sep 2022   Prob (F-statistic):                    0.000177
Time:                        22:31:07   Log-Likelihood:                         -9.1989
No. Observations:                   7   AIC:                                      24.40
Df Residuals:                       4   BIC:                                      24.24
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
mkt-rf         0.9972      0.076     13.113      0.000       0.786       1.208
smb            0.9531      0.257      3.714      0.021       0.241       1.665
hml           -0.2157      0.113     -1.914      0.128      -0.529       0.097
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.032
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.541
Skew:                          -0.571   Prob(JB):                        0.763
Kurtosis:                       2.258   Cond. No.                         5.08
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
res.params

mkt-rf    0.997204
smb       0.953057
hml      -0.215702
dtype: float64

In [9]:
res.resid

0    1.312685
1    0.865716
2   -0.124245
3   -1.467945
4   -0.504339
5    0.810496
6    0.349369
dtype: float64

# CS reg
- 第一步：按资产，时序回归，得到该资产对各因子的暴露beta
- 第二步：对资产的收益率取均值（y），与上一步的因子暴露一起（X），进行一次截面回归，得到因子收益率

In [10]:
test

,port,month,ret,mkt-rf,smb,hml
0,SL,202201,-13.3312,-6.25,-5.94,12.75
1,SL,202202,0.0517,-2.29,2.23,3.04
2,SL,202203,1.7806,3.05,-1.60,-1.80
3,SL,202204,-13.5805,-9.46,-1.41,6.19
4,SL,202205,-4.4206,-0.34,-1.85,8.41
5,SL,202206,-4.3163,-8.43,2.09,-5.97
6,SL,202207,13.4122,9.56,2.78,-4.08
7,SM,202201,-9.0569,-6.25,-5.94,12.75
8,SM,202202,1.2208,-2.29,2.23,3.04
9,SM,202203,1.0372,3.05,-1.60,-1.80


In [11]:
def np_ols(data, y, x):
    betas,_,_,_ = lstsq(data[x], data[y], rcond=None)
    return pd.Series(betas.flatten())

In [12]:
test['intercept'] = 1
beta = test.groupby('port').apply(np_ols, 'ret', ['intercept','mkt-rf','smb','hml'])
beta.columns = ['intercept','mkt-rf','smb','hml']
beta = beta.reindex(['SL','SM','SH','BL','BM','BH'])
beta

,intercept,mkt-rf,smb,hml
port,,,,
SL,0.226090,1.003225,0.925369,-0.235016
SM,0.355351,0.939081,1.116538,0.289688
SH,0.100432,0.982918,0.805102,0.756940
BL,0.455433,1.046054,-0.272175,-0.302630
BM,-0.361877,0.813760,0.269951,0.408317
BH,0.580463,1.065532,-0.151007,0.705807


In [13]:
y = test.groupby('port')[['ret']].mean()
y = y.reindex(['SL','SM','SH','BL','BM','BH'])
y

,ret
port,
SL,-2.914871
SM,-1.367186
SH,-0.308614
BL,-2.318257
BM,-1.069229
BH,0.374243


In [14]:
beta = beta.drop(columns=['intercept'])
res = sm.OLS(y, sm.add_constant(beta)).fit()
res.summary()

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ret   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.967
Method:                 Least Squares   F-statistic:                     50.36
Date:                Sat, 17 Sep 2022   Prob (F-statistic):             0.0195
Time:                        22:31:07   Log-Likelihood:                 3.8375
No. Observations:                   6   AIC:                            0.3250
Df Residuals:                       2   BIC:                           -0.5080
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.0902      1.173     -2.634      0.119      -8.139       1.959
mkt-rf         1.3638      1.161      1.175      0.361      -3.630       6.357
smb           -0.4966      0.178     -2.787      0.108      -1.263       0.270
hml            2.6468      0.221     11.975      0.007       1.696       3.598
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   3.209
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.414
Skew:                           0.542   Prob(JB):                        0.813
Kurtosis:                       2.307   Cond. No.                         27.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# FM reg

- 第一步：按资产，时序回归，得到该资产对各因子的暴露beta
- 第二步：对资产的收益率取均值（y），与上一步的因子暴露一起（X），进行t次截面回归，得到因子收益率时间序列

In [15]:
def np_ols(data, y, x):
    betas,_,_,_ = lstsq(data[x], data[y], rcond=None)
    return pd.Series(betas.flatten())

In [16]:
test['intercept'] = 1
X = test.groupby('port').apply(np_ols, 'ret', ['intercept','mkt-rf','smb','hml'])
X.columns = ['intercept','mkt-rf','smb','hml']
X = X.reindex(['SL','SM','SH','BL','BM','BH'])
X

,intercept,mkt-rf,smb,hml
port,,,,
SL,0.226090,1.003225,0.925369,-0.235016
SM,0.355351,0.939081,1.116538,0.289688
SH,0.100432,0.982918,0.805102,0.756940
BL,0.455433,1.046054,-0.272175,-0.302630
BM,-0.361877,0.813760,0.269951,0.408317
BH,0.580463,1.065532,-0.151007,0.705807


In [17]:
y = test[['port','month','ret']]
y = pd.merge(y, beta, how='left', on='port')
y['intercept'] = 1

In [18]:
lambd = y.groupby('month').apply(np_ols, 'ret', ['intercept','mkt-rf','smb','hml'])
lambd.columns = ['intercept','mkt-rf','smb','hml']
lambd

,intercept,mkt-rf,smb,hml
month,,,,
202201,-11.747505,6.250044,-5.867258,12.784370
202202,-2.854004,1.092186,2.258048,3.060879
202203,3.337016,-0.198232,-1.617914,-1.815918
202204,-3.065685,-6.606054,-1.519782,6.139422
202205,7.165082,-7.468933,-1.731737,8.392012
202206,-3.632889,-4.298447,2.209297,-5.952999
202207,-10.833527,20.776097,2.793358,-4.080461


In [19]:
lambd.mean(axis='rows')

intercept   -3.090216
mkt-rf       1.363809
smb         -0.496570
hml          2.646758
dtype: float64

# BYU-fiance

In [22]:
from scipy import stats

In [20]:
result = fama_macbeth(test,'month','ret',['mkt-rf','smb','hml'],intercept=True)
fm_summary(result)

,mean,std_error,tstat
intercept,0.008804,0.027242,0.323186
mkt-rf,0.359346,0.123803,2.902573
smb,0.009087,0.057107,0.159119
hml,-0.085871,0.115726,-0.742019


In [21]:
result

,intercept,mkt-rf,smb,hml
month,,,,
202201,-0.020297,0.126858,0.120566,-0.258790
202202,0.002580,-0.005908,0.005753,0.007843
202203,0.119615,0.364826,-0.191384,-0.215307
202204,-0.065206,0.616852,0.091941,-0.403627
202205,0.018267,-0.006211,-0.033793,0.153623
202206,-0.075332,0.635051,-0.157444,0.449734
202207,0.082003,0.783953,0.227970,-0.334574


In [24]:
stats.ttest_1samp(result['mkt-rf'],0)

Ttest_1sampResult(statistic=2.9025725030405676, pvalue=0.027242916364114175)